# Homework 7: Scraping part 3

## Part Three

Scrape the list of third party drivers license locations from https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations but include the link. Save as a CSV. Since it's more than just the text from the table, this requires actually using BeautifulSoup :(

Tips:

- You'll need user-agent headers for this one
- In class we always did something like item.find('h2') or the like because there was only ever one h2 we wanted. In this case there are four <td> tags that you want the text from! You'll need to use .find_all and treat them like normal lists ("give me the text from the first one," + "give me the text from the second one" etc etc)
- It's okay to have columns you don't want or need! You can always remove them later

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://travel-id-documents.az.gov/authorized-third-party-driver-license-locations"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

response = requests.get(url, headers=headers)

doc = BeautifulSoup(response.text)

In [3]:
items = doc.find(class_="table")

In [4]:
len(items)

2

In [5]:
items.find("thead").find("tr").find_all("td")

[<td><strong>Company</strong></td>,
 <td><strong>Address</strong></td>,
 <td><strong>Telephone</strong></td>,
 <td><strong>Hours</strong></td>]

In [6]:
headers = []
for item in items.find("thead").find("tr").find_all("td"):
    header = item.get_text()
    headers.append(header)

headers

['Company', 'Address', 'Telephone', 'Hours']

In [7]:
body = items.find("tbody").find_all("tr")
len(body)

40

In [8]:
body

[<tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
 <td>940 N. Alma School Rd., #104<br/>
 			Chandler, AZ 85224</td>
 <td>480.821.3288</td>
 <td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4:30 p.m.</td>
 </tr>,
 <tr><td><a href="http://az-mvd.com/" target="_blank">1 Stop Title &amp; Registration Services</a></td>
 <td>5036 W. Cactus Rd., Ste. 2<br/>
 			Glendale, AZ 85304</td>
 <td>602.264.2400</td>
 <td>Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4:30 p.m.</td>
 </tr>,
 <tr><td>Academy of Driving Motor Vehicle Center</td>
 <td>4733 E. Broadway Blvd.<br/>
 			Tucson, AZ 85711</td>
 <td>520.750.7572</td>
 <td>Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.</td>
 </tr>,
 <tr><td>Arizona Auto License</td>
 <td>1337 W. Prince Rd<br/>
 			Tucson, AZ 85705</td>
 <td>520.696.2023</td>
 <td>Driver License Hours: <br/>
 			Mon.-Fri. 9 a.m.-5 p.m.<br/>
 			Sat by Appt. Only</td>
 </tr>,
 <tr><td>Arizona Auto License Service LLC</td>
 <td>14

In [9]:
data = []

for row in body:
    cells = (row.find_all("td"))
    line = {}
    line["company"] = cells[0].get_text(strip=True)
    try:
        line["url"] = cells[0].find("a").get("href")
    except:
        pass
    line["address"] = cells[1].get_text(strip=True)
    line["telephone"] = cells[2].get_text(strip=True)
    line["hours"] = cells[3].get_text(strip=True)
    
    data.append(line)

len(data)    

40

In [10]:
data[0]

{'company': '1 Stop Title & Registration Services',
 'url': 'http://az-mvd.com/',
 'address': '940 N. Alma School Rd., #104Chandler, AZ 85224',
 'telephone': '480.821.3288',
 'hours': 'Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat.\xa09:00 a.m.-4:30 p.m.'}

In [11]:
df = pd.json_normalize(data)

In [12]:
df.head()

,company,url,address,telephone,hours
0,1 Stop Title & Registration Services,http://az-mvd.com/,"940 N. Alma School Rd., #104Chandler, AZ 85224",480.821.3288,Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 9:00 a.m.-4...
1,1 Stop Title & Registration Services,http://az-mvd.com/,"5036 W. Cactus Rd., Ste. 2Glendale, AZ 85304",602.264.2400,Mon.-Fri. 8:00 a.m.-6:00 p.m. Sat. 8:30 a.m.-4...
2,Academy of Driving Motor Vehicle Center,NaN,"4733 E. Broadway Blvd.Tucson, AZ 85711",520.750.7572,Mon.-Fri. 9 a.m.-5 p.m. and Sat. 9 a.m.-3 p.m.
3,Arizona Auto License,NaN,"1337 W. Prince RdTucson, AZ 85705",520.696.2023,Driver License Hours:Mon.-Fri. 9 a.m.-5 p.m.Sa...
4,Arizona Auto License Service LLC,NaN,"1457 N. Eliseo C Felix Jr.Way, Ste. 105 and 10...",623.925.5455 or Fax 623.925.5879,Mon.-Fri. 8 a.m.-5 p.m.


In [13]:
df.to_csv("arizona_transportation.csv")